In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from wordcloud import WordCloud

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/marenklan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marenklan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 19,000+ Products from Skinsort website

In [3]:
#Load csv review 
df= pd.read_csv('data/products.csv')


In [5]:
print(df.head())

                 brand                                      name   
0         The Ordinary          Glycolic Acid 7% Toning Solution  \
1       La Roche-Posay  Toleriane Hydrating Gentle Face Cleanser   
2         The Ordinary                Niacinamide 10% + Zinc 1%    
3  Youth To The People            Superfood Antioxidant Cleanser   
4                COSRX          Low pH Good Morning Gel Cleanser   

               type        country   
0             Toner         Canada  \
1     Face Cleanser         France   
2  Facial Treatment         Canada   
3     Face Cleanser  United States   
4     Face Cleanser    South Korea   

                                         ingridients   
0  Water,Glycolic Acid,Rosa Damascena Flower Wate...  \
1  Water,Glycerin,Pentaerythrityl Tetraethylhexan...   
2  Water,Niacinamide,Pentylene Glycol,Zinc PCA,Di...   
3  Water,Cocamidopropyl Hydroxysultaine,Sodium Co...   
4  Water,Cocamidopropyl Betaine,Sodium Lauroyl Me...   

                         

In [6]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19050 entries, 0 to 19049
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   brand        19050 non-null  object
 1   name         19050 non-null  object
 2   type         19035 non-null  object
 3   country      17218 non-null  object
 4   ingridients  18731 non-null  object
 5   afterUse     17526 non-null  object
dtypes: object(6)
memory usage: 893.1+ KB
None
             brand             name   type        country ingridients   
count        19050            19050  19035          17218       18731  \
unique        2224            18688     36             70       18171   
top     Neutrogena  Vitamin C Serum  Serum  United States      Cotton   
freq           204               23   2345           7315          17   

                  afterUse  
count                17526  
unique                2970  
top     Reduces Irritation  
freq                   265  


In [7]:
# missing values
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values)

Missing values per column:
 brand             0
name              0
type             15
country        1832
ingridients     319
afterUse       1524
dtype: int64


In [8]:
df.shape

(19050, 6)

In [9]:
df.columns

Index(['brand', 'name', 'type', 'country', 'ingridients', 'afterUse'], dtype='object')

In [14]:
df['type'].value_counts()

type
Serum                  2345
General Moisturizer    2149
Face Cleanser          1768
Sunscreen              1286
Bath & Body            1134
Toner                   967
Facial Treatment        861
Face Makeup             818
Eye Moisturizer         662
Exfoliator              586
Makeup Remover          564
Wet Mask                559
Sheet Mask              550
Other Haircare          460
Lip Moisturizer         421
Eye Makeup              416
Day Moisturizer         354
Oil                     354
Night Moisturizer       321
Essence                 298
Shampoo                 295
Tool                    250
Overnight Mask          198
Lip Makeup              191
Conditioner             184
Fragrance               169
Hand Care               156
Eye Mask                121
Makeup Applicator       115
Nail Care               110
Cheek Makeup            106
Tanning                 104
Lip Mask                 94
Emulsion                 65
False Eyelash             3
Other          

In [15]:
products_to_drop = ['Bath & Body', 'Face Makeup','Other Haircare','Eye Makeup','Shampoo','Tool','Lip Makeup','Conditioner','Fragrance','Makeup Applicator','Nail Care','Cheek Makeup','Tanning','False Eyelash','Other']
df_filtered = df[~df['type'].isin(products_to_drop)]


In [16]:
df_filtered.shape

(14694, 6)

In [17]:
print(df_filtered)

                     brand                                      name   
0             The Ordinary          Glycolic Acid 7% Toning Solution  \
1           La Roche-Posay  Toleriane Hydrating Gentle Face Cleanser   
2             The Ordinary                Niacinamide 10% + Zinc 1%    
3      Youth To The People            Superfood Antioxidant Cleanser   
4                    COSRX          Low pH Good Morning Gel Cleanser   
...                    ...                                       ...   
19045               CeraVe                 Hydrating Facial Cleanser   
19046     Beauty of Joseon                     Ginseng Essence Water   
19047               CeraVe             PM Facial Moisturizing Lotion   
19048         The Ordinary         AHA 30% + BHA 2% Peeling Solution   
19049                COSRX     Advanced Snail 96 Mucin Power Essence   

                    type        country   
0                  Toner         Canada  \
1          Face Cleanser         France   
2     

In [19]:
df_filtered.duplicated().value_counts()

False    14685
True         9
Name: count, dtype: int64

In [20]:
# remove duplicates
df_filt = df_filtered.drop_duplicates()
# reset index inplace
df_filt.reset_index(inplace=True, drop=True)
df_filt.head(5)

,brand,name,type,country,ingridients,afterUse
0,The Ordinary,Glycolic Acid 7% Toning Solution,Toner,Canada,"Water,Glycolic Acid,Rosa Damascena Flower Wate...","Good For Oily Skin,Skin Texture,Reduces Large ..."
1,La Roche-Posay,Toleriane Hydrating Gentle Face Cleanser,Face Cleanser,France,"Water,Glycerin,Pentaerythrityl Tetraethylhexan...","Good For Oily Skin,Redness Reducing,Reduces Ir..."
2,The Ordinary,Niacinamide 10% + Zinc 1%,Facial Treatment,Canada,"Water,Niacinamide,Pentylene Glycol,Zinc PCA,Di...","Good For Oily Skin,Redness Reducing,Acne Fight..."
3,Youth To The People,Superfood Antioxidant Cleanser,Face Cleanser,United States,"Water,Cocamidopropyl Hydroxysultaine,Sodium Co...","Redness Reducing,Reduces Irritation,Skin Textu..."
4,COSRX,Low pH Good Morning Gel Cleanser,Face Cleanser,South Korea,"Water,Cocamidopropyl Betaine,Sodium Lauroyl Me...","Good For Oily Skin,Reduces Irritation,Reduces ..."


In [21]:
# missing values
missing_values = df_filt.isnull().sum()
print("Missing values per column:\n", missing_values)

Missing values per column:
 brand             0
name              0
type             15
country        1240
ingridients      35
afterUse        887
dtype: int64


In [27]:
df_miss = df_filt.dropna(subset=['ingridients','type','afterUse'])


In [28]:
# missing values
missing_values = df_miss.isnull().sum()
print("Missing values per column:\n", missing_values)

Missing values per column:
 brand             0
name              0
type              0
country        1171
ingridients       0
afterUse          0
dtype: int64


In [29]:
df_miss.shape

(13784, 6)

In [30]:
# Rename a column in-place
df_miss.rename(columns={'ingridients': 'ingredients'}, inplace=True)

/var/folders/tl/40517jl91c37d_b8j15w_1wm0000gn/T/ipykernel_4796/4197624215.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_miss.rename(columns={'ingridients': 'ingredients'}, inplace=True)


In [31]:
df_miss.head()

,brand,name,type,country,ingredients,afterUse
0,The Ordinary,Glycolic Acid 7% Toning Solution,Toner,Canada,"Water,Glycolic Acid,Rosa Damascena Flower Wate...","Good For Oily Skin,Skin Texture,Reduces Large ..."
1,La Roche-Posay,Toleriane Hydrating Gentle Face Cleanser,Face Cleanser,France,"Water,Glycerin,Pentaerythrityl Tetraethylhexan...","Good For Oily Skin,Redness Reducing,Reduces Ir..."
2,The Ordinary,Niacinamide 10% + Zinc 1%,Facial Treatment,Canada,"Water,Niacinamide,Pentylene Glycol,Zinc PCA,Di...","Good For Oily Skin,Redness Reducing,Acne Fight..."
3,Youth To The People,Superfood Antioxidant Cleanser,Face Cleanser,United States,"Water,Cocamidopropyl Hydroxysultaine,Sodium Co...","Redness Reducing,Reduces Irritation,Skin Textu..."
4,COSRX,Low pH Good Morning Gel Cleanser,Face Cleanser,South Korea,"Water,Cocamidopropyl Betaine,Sodium Lauroyl Me...","Good For Oily Skin,Reduces Irritation,Reduces ..."


In [32]:
df_miss['brand'].value_counts()

brand
Neutrogena            187
La Roche-Posay        144
innisfree             132
Paula's Choice        120
Missha                106
                     ... 
Lubriderm               1
Dermabien               1
Eskinol                 1
Function Of Beauty      1
Plants are Purple       1
Name: count, Length: 1794, dtype: int64

## Products Sephora 